# MLRun secret handling using Vault - Demo

This notebook demonstrates secret creation and handling in MLRun.


In [ ]:
from mlrun import new_task, get_run_db, new_function, run_local
import os
from mlrun import mlconf, code_to_function, new_project

## Create a project & initialize Vault support
When a project is created, the `create_project_secrets` command can be used to request that the underlying framework is created that will enable Vault secrets to be used with this project. Calling this method on the project will create the following constructs for the project (if not already existing):

1. A k8s serviceaccount (`sa-vault-{project name}`)
2. A Vault policy (`mlrun-project-{project name}`) that enables access to secrets in the project path (`/secrets/secret/mlrun/projects/{proj name}`)
3. A Vault k8s role (`mlrun-role-project-{project name}`) that associates the SA's token with the policy

These configurations are performed on the MLRun API server side, not from the client.

In [ ]:
proj_name = 'vault-mlrun'
func_name = 'vault-func'

proj = new_project(proj_name)

The `create_vault_secrets` utility function allows creating project secrets, and will place them in the project secret path.
>Note: This method runs on the MLRun API server. For running similar commands locally (for example, if you have no MLRun API setup), please refer to the instructions later in this notebook.

In [ ]:
project_secrets = {'aws_key': '1234567890', 'github_key': 'proj1Key!!!'}
proj.create_vault_secrets(project_secrets)
proj.get_vault_secret_keys()

Project objects also allow access to Vault secrets directly (not through function contexts). You need to associate the project with Vault secret-source using the `.with_secret()` function, and then can use the `.get_secret()` utility function to extract secret value.
This allows client-side, read-only access to vault secrets associated with the given project. To perform write operations, refer to next cell.

>Note: These methods runs on client-side, which means the computer/pod hosting this notebook must have Vault connectivity (and access Token) for this to work.

In [ ]:
# Using empty list for secret keys grants access to all secrets associated with this project.
proj.with_secrets('vault',[])
print(proj.get_vault_secret_keys())
print(proj.get_secret('github_key'))

### Accessing Vault functionality locally
The following commands allow client-side write (and read) commands on secrets, both in project context and in user context (instead of `project` parameter use the `user` parameter). Of course, these APIs are still subject to Vault access permissions for the running process, so they do not allow elevated permissions beyond those granted to the execution context.

In [ ]:
from mlrun.utils.vault import VaultStore

vault = VaultStore()

project_secrets["azure_key"] = "1111-2222-3333-4444"
vault.add_vault_secrets(project_secrets, project=proj_name)
vault.get_secrets([],project=proj_name)

## Create and deploy a function

In [ ]:
func_path = os.path.abspath("vault_function.py")

func = code_to_function(name=func_name,
                        filename=func_path,
                        handler='vault_func',
                        project=proj_name,
                        kind='job')

func.doc()

In [ ]:
# If needed, use this command to utilize a local MLRun image for your functions, rather than the default mlrun/mlrun image.
# func.spec.image = 'saarcoiguazio/mlrun:unstable'

## Initialize function runtime and execute
The `.with_secrets` function has a '`vault`' secret kind that will pass the specified Vault project secrets to the function context. The function spec
only contains the keys of the secrets ('aws_key' etc.) - the actual secret value is retrieved from Vault and planted in the function 
context in runtime.

In [ ]:
task = new_task(project=proj_name,
                name='vault_test_run',
                handler='vault_func',
                params={'secrets':['password', 'github_key', 'aws_key']})

# Add access to project-level secrets
# task.with_secrets('vault', ["aws_key", "github_key"])
task.with_secrets('vault', [])

# Can also use the following code to add user-level secrets
# task.with_secrets('vault', {"user": "admin", "secrets": ["password", "user_id"]})

print(task.to_yaml())

In [ ]:
result = func.run(task)

## Run the same function in another project's context
We will create a 2nd project, and assign different secret values to it. When the same function is executed in the new project's runtime context, it will get 
the new project's secrets. When running in this context, the function has no access to other projects' secrets.

In [ ]:
proj_name_2 = 'vault-mlrun-2'
proj2 = new_project(proj_name_2)
proj2.create_vault_secrets({'aws_key': '0987654321', 'github_key': 'proj2Key???', 'password': 'myPassword'})

In [ ]:
task2 = new_task(project=proj_name_2,
                 name='vault_test_run_2',
                 handler='vault_func',
                 params={'secrets':['password', 'github_key', 'aws_key']})
task2.with_secrets('vault', ["aws_key", "github_key", "password"])

result = func.run(task2)

In [ ]:
proj2.get_vault_secret_keys()